# Task
In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from [Kaggle](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon). 

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

**Note:** You will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). 
You can also use a computer without a GPU (e.g. your laptop), but it will be slower.

In [27]:
import os
import shutil

import numpy as np
import matplotlib.pyplot as plt

In [28]:
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [29]:
import tensorflow
tensorflow.__version__

'2.10.0'

# Data preparation
The dataset contains around 1900 images of dinos and around 1900 images of dragons. 

The dataset contains separate folders for training and validation. 

In [30]:
train_dir = 'data/train'
val_dir = 'data/test'

# Model
For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [31]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3),
                        activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [32]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8), 
              metrics=['acc'])

# Question 1
Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy` [X]
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

# Question 2
What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873 [X]
- 14215873
- 19215873

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 175232)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11214912  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

# Generators and Training
For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and validation 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
```

In [34]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [35]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    shuffle=True,
                                                    class_mode='binary')

Found 1594 images belonging to 2 classes.


In [36]:
val_generator = val_datagen.flow_from_directory(test_dir,
                                                target_size=(150, 150),
                                                batch_size=20,
                                                shuffle=True,
                                                class_mode='binary')

Found 394 images belonging to 2 classes.


In [37]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


In [38]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator)

Epoch 1/10
80/80 [==============================] - 15s 177ms/step - loss: 0.5568 - acc: 0.7077 - val_loss: 0.4166 - val_acc: 0.8401
Epoch 2/10
80/80 [==============================] - 14s 169ms/step - loss: 0.4078 - acc: 0.8287 - val_loss: 0.3628 - val_acc: 0.8579
Epoch 3/10
80/80 [==============================] - 14s 169ms/step - loss: 0.3382 - acc: 0.8651 - val_loss: 0.3401 - val_acc: 0.8579
Epoch 4/10
80/80 [==============================] - 14s 172ms/step - loss: 0.2883 - acc: 0.8883 - val_loss: 0.3060 - val_acc: 0.8832
Epoch 5/10
80/80 [==============================] - 14s 171ms/step - loss: 0.2744 - acc: 0.8864 - val_loss: 0.2913 - val_acc: 0.8680
Epoch 6/10
80/80 [==============================] - 14s 172ms/step - loss: 0.2243 - acc: 0.9115 - val_loss: 0.2750 - val_acc: 0.8807
Epoch 7/10
80/80 [==============================] - 14s 170ms/step - loss: 0.2003 - acc: 0.9310 - val_loss: 0.2694 - val_acc: 0.8807
Epoch 8/10
80/80 [==============================] - 14s 170ms/step - 

In [39]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Question 3
What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90 [X]
- 0.20


In [40]:
acc_median = np.median(acc)
acc_median

0.8999372720718384

# Question 4
What is the standard deviation of training loss for all the epochs for this model?

- 0.11 [X]
- 0.66
- 0.99
- 0.33

In [41]:
loss_std = np.std(loss)
loss_std

0.1241493000804223

# Data Augmentation
For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [42]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Question 5
Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of validation loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

# Question 6
What's the average of validation accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24